In [5]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
!ls -lt '/content/gdrive/My Drive/' 
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import binary_accuracy, AUC
from sklearn.utils import shuffle

Mounted at /content/gdrive
total 6673657
-rw------- 1 root root        107 Jan 16 12:16  checkpoint
-rw------- 1 root root    6396921 Jan 15 13:16  BBH_Classification_4.ckpt.data-00000-of-00001
-rw------- 1 root root       4061 Jan 15 13:16  BBH_Classification_4.ckpt.index
-rw------- 1 root root    6478872 Jan 15 13:16  BBH_Classification_Model_4.h5
drwx------ 2 root root       4096 Jan 14 07:25 'FYP NBs'
-rw------- 1 root root    3767216 Jan 13 09:10  BBH_Classification_Model_3.h5
-rw------- 1 root root       3261 Jan 13 09:10  BBH_Classification_3.ckpt.index
-rw------- 1 root root    3701371 Jan 13 09:10  BBH_Classification_3.ckpt.data-00000-of-00001
-rw------- 1 root root       3261 Jan 12 13:14  BBH_Classification_2.ckpt.index
-rw------- 1 root root    3701511 Jan 12 13:14  BBH_Classification_2.ckpt.data-00000-of-00001
-rw------- 1 root root    3766752 Jan 12 13:14  BBH_Classification_Model_2.h5
-rw------- 1 root root   28299544 Jan 11 10:44  BBH_Classification_Model_1.h5
-rw------

In [18]:
def model_creation():
  n_timesteps, n_features = 16384, 1
  input_shape=(n_timesteps,n_features)
  model=Sequential()

  model.add(Conv1D(filters=8, kernel_size=8, activation='relu', input_shape=input_shape))
  # model.add(Conv1D(8, kernel_size=8, activation='relu'))
  model.add(MaxPooling1D(pool_size=4))

  model.add(Conv1D(16, kernel_size=8, activation='relu'))
  # model.add(Conv1D(16, kernel_size=8, activation='relu'))
  model.add(MaxPooling1D(pool_size=4))

  model.add(Conv1D(32, kernel_size=8, activation='relu'))
  # model.add(Conv1D(64, kernel_size=8, activation='relu'))
  model.add(MaxPooling1D(pool_size=4))

  # model.add(Conv1D(64, kernel_size=8, activation='relu'))
  # model.add(Conv1D(128, kernel_size=8, activation='relu'))
  # model.add(MaxPooling1D(pool_size=4))

  model.add(Flatten())
  # model.add(GlobalAveragePooling1D(data_format='channels_last'))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.3))

  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.3))

  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.3))

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer= Adam(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08),
                loss='binary_crossentropy',
                metrics=['binary_accuracy', 'accuracy'])
  model.summary()
  return model

In [19]:
model = model_creation()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_11 (Conv1D)          (None, 16377, 8)          72        
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 4094, 8)          0         
 g1D)                                                            
                                                                 
 conv1d_12 (Conv1D)          (None, 4087, 16)          1040      
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 1021, 16)         0         
 g1D)                                                            
                                                                 
 conv1d_13 (Conv1D)          (None, 1014, 32)          4128      
                                                                 
 max_pooling1d_13 (MaxPoolin  (None, 253, 32)         

In [20]:
# training file locations
postrainfoldername = '/content/gdrive/MyDrive/Positive_Training_DATA'
negtrainfoldername = '/content/gdrive/MyDrive/Negative_Train_DATA'

# postive and negative prefixes
postrainprefix = 111100000
negtrainprefix = 101100000

#samples of pos and neg training data
postrainsamples = 24576
negtrainsamples = 8192

# initialises global variables 
def initialise():
  global postrainprefix
  global negtrainprefix
  postrainprefix = 111100000
  negtrainprefix = 101100000

# number of batches
batch_looping_times = 64

# number of samples in each batch
samples = (postrainsamples//batch_looping_times) +(negtrainsamples//batch_looping_times)

print(samples)
y=np.concatenate((np.ones(((postrainsamples//batch_looping_times), 1)), np.zeros(((negtrainsamples//batch_looping_times), 1))))
print(y.shape)

512
(512, 1)


In [21]:
def batchload_train():
  global y
  global postrainprefix
  global negtrainprefix
  # print(samples)
  X=np.zeros((samples,8192*2))
  # y=np.zeros((samples,1))
  for i in range(postrainsamples//batch_looping_times):
    posfile_name = postrainfoldername+'/'+str(postrainprefix+i+1)+'.npy'
    X[i] = np.load(posfile_name)
    # y[i] = 1
  for i in range(negtrainsamples//batch_looping_times):
    negfile_name = negtrainfoldername+'/'+str(negtrainprefix+i+1)+'.npy'
    X[i+(postrainsamples//batch_looping_times)] = np.load(negfile_name)
    # y[i] = 0

  X = X.reshape(samples, 16384,1)
  return X, y



In [22]:
def get_data():
  global postrainprefix
  global negtrainprefix
  global y
  initialise()
  while True:

    trainX, trainy = batchload_train()
    # train_dataset = tf.data.Dataset.from_tensor_slices((trainX, trainy)).shuffle(729)
    # train_dataset = train_dataset.batch(batch_size)
    trainX, trainy = shuffle(trainX, trainy, random_state=53)
    postrainprefix += (postrainsamples//batch_looping_times)
    negtrainprefix += (negtrainsamples//batch_looping_times)
    if (postrainprefix >= 111124575):
      initialise()
    yield trainX, trainy




In [23]:

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_path = "/content/gdrive/My Drive/BBH_Classification_5.ckpt"


# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=0, 
    save_weights_only=True,
    save_freq = 1)



In [28]:
from keras.models import load_model
def model_training():
    global model
    model = load_model("/content/gdrive/MyDrive/BBH_Classification_Model_5.h5")
    model.fit(get_data(), epochs=2, steps_per_epoch=batch_looping_times, verbose = 1, shuffle=
                        True, callbacks=[cp_callback], batch_size = 512, use_multiprocessing = True)
    model.save("/content/gdrive/MyDrive/BBH_Classification_Model_5.h5")

In [29]:
# model 5
model_training()

Epoch 1/2
64/64 [==============================] - 327s 5s/step - loss: 0.0626 - binary_accuracy: 0.9815 - accuracy: 0.9815
Epoch 2/2
64/64 [==============================] - 305s 5s/step - loss: 0.0539 - binary_accuracy: 0.9853 - accuracy: 0.9853


In [27]:
# model 5
model_training()

Epoch 1/2
64/64 [==============================] - 330s 5s/step - loss: 0.0634 - binary_accuracy: 0.9825 - accuracy: 0.9825
Epoch 2/2
64/64 [==============================] - 309s 5s/step - loss: 0.0766 - binary_accuracy: 0.9757 - accuracy: 0.9757


In [25]:
# model 5
model_training()

Epoch 1/10
64/64 [==============================] - 320s 5s/step - loss: 1.1930 - binary_accuracy: 0.7151 - accuracy: 0.7151
Epoch 2/10
64/64 [==============================] - 295s 5s/step - loss: 0.3462 - binary_accuracy: 0.7686 - accuracy: 0.7686
Epoch 3/10
64/64 [==============================] - 306s 5s/step - loss: 0.2612 - binary_accuracy: 0.8985 - accuracy: 0.8985
Epoch 4/10
64/64 [==============================] - 296s 5s/step - loss: 0.2026 - binary_accuracy: 0.9225 - accuracy: 0.9225
Epoch 5/10
64/64 [==============================] - 308s 5s/step - loss: 0.1479 - binary_accuracy: 0.9455 - accuracy: 0.9455
Epoch 6/10
64/64 [==============================] - 300s 5s/step - loss: 0.1168 - binary_accuracy: 0.9615 - accuracy: 0.9615
Epoch 7/10
64/64 [==============================] - 311s 5s/step - loss: 0.1022 - binary_accuracy: 0.9668 - accuracy: 0.9668
Epoch 8/10
64/64 [==============================] - 303s 5s/step - loss: 0.0935 - binary_accuracy: 0.9703 - accuracy: 0.9703


In [ ]:
# model 4
model_training()

Epoch 1/10
64/64 [==============================] - 606s 9s/step - loss: 0.5864 - binary_accuracy: 0.7643 - accuracy: 0.7643
Epoch 2/10
64/64 [==============================] - 588s 9s/step - loss: 0.2632 - binary_accuracy: 0.8592 - accuracy: 0.8592
Epoch 3/10
64/64 [==============================] - 594s 9s/step - loss: 0.2024 - binary_accuracy: 0.9104 - accuracy: 0.9104
Epoch 4/10
64/64 [==============================] - 587s 9s/step - loss: 0.1558 - binary_accuracy: 0.9370 - accuracy: 0.9370
Epoch 5/10
64/64 [==============================] - 588s 9s/step - loss: 0.1278 - binary_accuracy: 0.9508 - accuracy: 0.9508
Epoch 6/10
64/64 [==============================] - 588s 9s/step - loss: 0.1101 - binary_accuracy: 0.9597 - accuracy: 0.9597
Epoch 7/10
64/64 [==============================] - 587s 9s/step - loss: 0.0978 - binary_accuracy: 0.9648 - accuracy: 0.9648
Epoch 8/10
64/64 [==============================] - 589s 9s/step - loss: 0.0801 - binary_accuracy: 0.9715 - accuracy: 0.9715
